In [11]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

In [12]:
# Load the data
pd.set_option('display.max_columns', None)
ais_test = pd.read_csv('ais_test.csv')
ais_train = pd.read_csv('first_50000_rows.csv', sep='|')

# Ensure the 'time' column is in datetime format
ais_train['time'] = pd.to_datetime(ais_train['time'])

# Get a list of vessel IDs from the ais_test dataset
test_vessels = ais_test['vesselId'].unique()

# Group by 'vesselId' and get the row with the most recent 'time' for each vessel
most_recent_rows = ais_train.loc[ais_train.groupby('vesselId')['time'].idxmax()]

# Filter most_recent_rows to include only vessels present in ais_test
most_recent_rows = most_recent_rows[most_recent_rows['vesselId'].isin(test_vessels)]

# Display the result
print(most_recent_rows)

                     time    cog   sog  rot  heading  navstat       etaRaw  \
46220 2024-01-04 23:19:05  244.0  13.2    0      246        0  01-06 11:30   
40859 2024-01-04 11:56:06  184.5   9.4  -11      184        0  01-18 08:00   
44705 2024-01-04 20:16:41   99.7  17.4   -6       99        0  01-07 16:00   
28744 2024-01-03 10:04:59  215.0  10.5 -127      206        0  01-02 06:00   
49950 2024-01-05 07:18:20  267.0   9.4    0      268        0  01-04 11:00   
...                   ...    ...   ...  ...      ...      ...          ...   
39963 2024-01-04 09:54:35  339.9  12.6    0      336        0  01-06 05:00   
45277 2024-01-04 21:33:14  120.6   9.6    0      118        0  12-04 06:00   
11177 2024-01-01 23:44:37  211.3   0.0    0      203        5  12-28 17:00   
49936 2024-01-05 07:18:18  172.2  18.7    0      170        0  01-05 14:00   
49862 2024-01-05 07:17:20    0.0   0.0    1      134        5  01-05 05:00   

       latitude  longitude                   vesselId  \
46220 

In [13]:
def visualize_latest_vessel_positions(df):
    """
    Visualize the most recent positions of vessels on a map with markers for each vessel.

    Parameters:
    - df (pandas.DataFrame): A DataFrame with columns ['time', 'latitude', 'longitude', 'vesselId'].

    Returns:
    - A Plotly interactive figure.
    """
    # Ensure 'time' is in datetime format for better tooltip handling
    df['time'] = pd.to_datetime(df['time'])
    
    # Define a color palette
    color_map = px.colors.qualitative.Plotly

    # Mapping each vessel ID to a color
    unique_vessels = df['vesselId'].unique()
    colors = {vessel_id: color_map[i % len(color_map)] for i, vessel_id in enumerate(unique_vessels)}

    # Create the base map with markers for the most recent positions
    fig = go.Figure()

    # Add markers for each vessel's latest data point
    for vessel_id in unique_vessels:
        vessel_data = df[df['vesselId'] == vessel_id]
        fig.add_trace(go.Scattergeo(
            lon=vessel_data['longitude'],
            lat=vessel_data['latitude'],
            mode='markers',
            marker=dict(
                size=8,
                color=colors[vessel_id],
                opacity=0.8,
                line=dict(width=1, color='DarkSlateGrey')
            ),
            name=f'Vessel {vessel_id}',
            hoverinfo='text',
            text=vessel_data.apply(lambda row: f'ID: {vessel_id}<br>Time: {row["time"]}<br>Lat: {row["latitude"]:.3f}<br>Lon: {row["longitude"]:.3f}', axis=1)
        ))

    # Enhancing map and layout details
    fig.update_geos(fitbounds="locations", showcountries=True, countrycolor="RebeccaPurple")
    fig.update_layout(margin={"r":0,"t":30,"l":0,"b":0},
                      title='Most Recent Vessel Positions',
                      title_font_size=20)
    
    return fig

# Assuming 'most_recent_rows' contains the latest vessel positions
visualize_latest_vessel_positions(most_recent_rows)